In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta



StatementMeta(, 122bdaa7-0ca4-4de9-83c5-ed249be507af, 5, Finished, Available, Finished)

### Chargement Initiale 

In [9]:
def build_data_url(date):
    return "https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/" + date + "_wind_power_data.csv"


initial_date = "20240615"
initial_url = build_data_url(initial_date)
print(f"Lien du fichier initiale : {initial_url}")

StatementMeta(, 122bdaa7-0ca4-4de9-83c5-ed249be507af, 11, Finished, Available, Finished)

Lien du fichier initiale : https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/20240615_wind_power_data.csv


In [2]:
df = pd.read_csv(initial_url)
df["date"] = pd.to_datetime(df["date"])

print("Nombre de lignes : ", len(df))
df.head(5)

StatementMeta(, f7d50b7b-5de9-490f-8604-0cbb82e2f52b, 4, Finished, Available, Finished)

Lien du fichier initiale : https://raw.githubusercontent.com/gsoulat/data-training-fabric/main/eolienne/20240615_wind_power_data.csv
Nombre de lignes :  432


,production_id,date,time,turbine_name,capacity,location_name,latitude,longitude,region,status,responsible_department,wind_speed,wind_direction,energy_produced
0,6049,2024-06-15,00-00-00,Turbine A,2200,Location 1,34.0522,-118.2437,Region A,Online,Operations,5.74042,SW,1783.39373
1,6050,2024-06-15,00-00-00,Turbine B,2000,Location 2,36.7783,-119.4179,Region B,Inspection,Maintenance,23.92376,E,0.00000
2,6051,2024-06-15,00-00-00,Turbine C,2500,Location 3,40.7128,-74.0060,Region C,Online,Operations,5.72221,NW,1651.83735
3,6052,2024-06-15,00-10-00,Turbine A,2200,Location 1,34.0522,-118.2437,Region A,Online,Operations,13.01549,E,1351.88077
4,6053,2024-06-15,00-10-00,Turbine B,2000,Location 2,36.7783,-119.4179,Region B,Online,Operations,8.76995,E,931.99157


In [2]:
# Chemin vers la table Bronze 

# les ids sont dispo dans l'url de la page du lakehouse. Example :
# https://app.fabric.microsoft.com/groups/79daad76-9b51-4f79-b1a3-5049702b5055/lakehouses/c9815a16-8f35-44be-8356-61d5566edb18
workspace_id = "79daad76-9b51-4f79-b1a3-5049702b5055" 
lakehouse_id = "c9815a16-8f35-44be-8356-61d5566edb18" 

bronze_table_path = f"abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}/Tables/dbo/wind_power"


StatementMeta(, 122bdaa7-0ca4-4de9-83c5-ed249be507af, 4, Finished, Available, Finished)

In [4]:
df_spark = spark.createDataFrame(df)
df_spark.write.format("delta").mode("overwrite").save(bronze_table_path)

# alternative :
# df_spark.write.format("delta").mode("overwrite").saveAsTable("wind_power")

StatementMeta(, f7d50b7b-5de9-490f-8604-0cbb82e2f52b, 6, Finished, Available, Finished)

Verification des données

In [4]:
df_read = spark.read.format("delta").load(bronze_table_path).toPandas()

# alternative : 
# df_read = spark.read.format("delta").table("wind_power").toPandas()

print("Nombre de lignes : ", len(df_read))
print("colonnes : ", df_read.columns)
df_read.head(5)


StatementMeta(, 122bdaa7-0ca4-4de9-83c5-ed249be507af, 6, Finished, Available, Finished)

Nombre de lignes :  432
colonnes :  Index(['production_id', 'date', 'time', 'turbine_name', 'capacity',
       'location_name', 'latitude', 'longitude', 'region', 'status',
       'responsible_department', 'wind_speed', 'wind_direction',
       'energy_produced'],
      dtype='object')


,production_id,date,time,turbine_name,capacity,location_name,latitude,longitude,region,status,responsible_department,wind_speed,wind_direction,energy_produced
0,6049,2024-06-15,00-00-00,Turbine A,2200,Location 1,34.0522,-118.2437,Region A,Online,Operations,5.74042,SW,1783.39373
1,6050,2024-06-15,00-00-00,Turbine B,2000,Location 2,36.7783,-119.4179,Region B,Inspection,Maintenance,23.92376,E,0.00000
2,6051,2024-06-15,00-00-00,Turbine C,2500,Location 3,40.7128,-74.0060,Region C,Online,Operations,5.72221,NW,1651.83735
3,6052,2024-06-15,00-10-00,Turbine A,2200,Location 1,34.0522,-118.2437,Region A,Online,Operations,13.01549,E,1351.88077
4,6053,2024-06-15,00-10-00,Turbine B,2000,Location 2,36.7783,-119.4179,Region B,Online,Operations,8.76995,E,931.99157


Import de la date suivante

In [5]:
max_date =pd.to_datetime(spark.read.format("delta").load(bronze_table_path).toPandas()["date"], format="%Y%m%d").max()

next_date = (max_date + timedelta(days=1)).strftime("%Y%m%d")

print(max_date, next_date)

StatementMeta(, 122bdaa7-0ca4-4de9-83c5-ed249be507af, 7, Finished, Available, Finished)

2024-06-15 00:00:00 20240616


In [ ]:
next_day_url = build_data_url(next_date)
